In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from datetime import date

from sql_driver import SQLDriver

from mdx_driver import execute_query

pd.options.mode.chained_assignment = None

In [4]:
mov_driver = SQLDriver("MOVOfGoodsDynamic")
dbd_driver = SQLDriver("DBDMart")
sta_driver = SQLDriver("StagingArea")
eab_driver = SQLDriver(db="EasyBuying", server="easybuying-tsum")
dbd_driver2 = SQLDriver(db='DBDMart',server='DataMart-VDI')

In [5]:
EXCHANGE_RATE = 110

HUGO_BOSS_RRP_PATH = r"\\mercury.ru\dfspublic$\ЦУМ\Аналитический отдел\_by Names\~ADSmirnov\Ценообразование\Hugo Boss РРЦ.xlsx"

In [6]:
df_costs = mov_driver.read("""
    select*
    from dbdmart.[SalesDevelopment].cost_price_all
    where itemid <> '2152002'
""")
df_costs

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Марка,Субмарка,itemid,_ktt1,_ktt2,_ktt3,_ktt3id,analog_code,axarticle,vendor_type,seasonkey,seasonid,cost_price_delivery_eur,cost_price_delivery_rur
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,67,SS 2022,28.0,2393.0500
1,Givenchy,Givenchy,5570765,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,bm713q/3y6b,BM713Q3Y6B,ВнПОСТ_ЦУМ,65,SS 2021,204.0,20782.0300
2,Armani,Armani Junior,1881322,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,M4B062H,M4B06/2H/12A-16A,РПОСТ_ЦУМ,32,SS 2010,187.5,16195.0000
3,MONNALISA,Monnalisa Bebe,4230284,Товары для детей,Товары для новорожденных,Одежда,KTT0201758,314704/4914,314704/4914,ВнПОСТ_ЦУМ,47,FW 14/15,38.5,6994.9990
4,John Galliano Junior,John Galliano Junior,4029528,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,AWR/M1,00WAWR/JAAM1/10-14,РПОСТ_ЦУМ,41,SS 2012,56.9,6588.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519146,Firetrap,Firetrap,1831698,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,fgdr114a/skull_raiser,FGDR114A/SKULL_RAISER,РПОСТ_ЦУМ,34,FW 09/10,14.0,0.0000
1519147,Acne,Acne Studios,5111696,Товары для женщин,Аксессуары,Головные уборы,KTT0100662,17a164/hoy hat chunky,17A164,ВнПОСТ_ЦУМ,53,FW 16/17,46.0,4078.1600
1519148,Two women in the world,Two Women In The World,5093093,Товары для женщин,Одежда,Одежда джинсовая,KTT0101901,texsana lc4mc,TEXANA/LC4MC,ВнПОСТ_ЦУМ,53,FW 16/17,68.0,6530.5500
1519149,Moschino,Moschino,5001447,Товары для мужчин,Одежда,"Сорочки, рубашки",KTT0103078,0206/5261,J0206/5261,ВнПОСТ_ЦУМ,50,FW 15/16,131.0,10.0000


In [7]:
df_base = execute_query(
    """
    SELECT NON EMPTY
    {
    [Measures].[Delivery PCS]
    --, [Measures].[Conf PCS]
    } ON COLUMNS, NON EMPTY

    { (
    [Товары].[Analog Code].[Analog Code]
    * [Товары].[KTTID Level 03].[KTTID Level 03]
    * [Товары].[Базовая цена].[Базовая цена]
    * [Сезоны].[Сезон].[Сезон]
    ) }
    ON ROWS

    FROM ( SELECT (
    { [Формы собственности].[Форма собственности].&[1] 
    }
    ,{
    [Тип заказа].[Тип заказа].[Retail]
    ,[Тип заказа].[Тип заказа].[Consignation]
    ,[Тип заказа].[Тип заказа].[Internet]
    ,[Тип заказа].[Тип заказа].[Re-Order]
    ,[Тип заказа].[Тип заказа].[NS]
    },
    -{[Товары].[Базовая цена].&[0]},
    -{[Товары].[ItemID].[Unknown]}
    )

    ON COLUMNS FROM [Analyse])
    """, 
    add_hierarchy=True,
)
df_base.columns = ["analog_code", "ktt3id", "base_cost", "season", "Delivery PCS"]
df_base.base_cost = df_base.base_cost.astype(float)
df_base = df_base.drop_duplicates(subset=["analog_code", "ktt3id"], keep="last")
df_base

Timer: 0:00:28


,analog_code,ktt3id,base_cost,season,Delivery PCS
0,#0002,KTT0102250,398.4,SS 2023,14
1,#0006,KTT0200093,398.4,SS 2023,14
2,#0007,KTT0102250,490.4,SS 2023,16
3,#0011,KTT0102250,306.4,SS 2023,12
4,#0012,KTT0201081,134.8,SS 2023,6
...,...,...,...,...,...
4950,ZAYN/OP,KTT0101893,361.0,SS 2023,2
4951,ZO31/6803,KTT0101909,315.0,FW 23/24,12
4952,ZR35001/63D4,KTT0101894,297.0,FW 23/24,42
4953,ZR85001/63B5,KTT0101894,222.0,FW 23/24,24


In [8]:
def active_mu(row):
    if row.mu_article > 0:
        return row.mu_article
    return row.mu_old

In [9]:
df_mus = mov_driver.read("""
    select * from DBDMart.SalesDevelopment.cost_price_all_mu
""")
df_mus["mu"] = df_mus.apply(lambda x: active_mu(x), axis=1)
df_mus

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Субмарка,itemid,_ktt3id,seasonkey,mu_old,mu_current,mu_article,mu
0,ARMANI JUNIOR,5159016,KTT0201647,55,2.50,0.00,0.0,2.50
1,DOLCE&GABBANA,5188878,KTT0201704,56,2.65,3.53,0.0,2.65
2,GIVENCHY,5624353,KTT0101901,67,2.90,2.90,0.0,2.90
3,MSGM,5587341,KTT0101898,66,3.06,3.24,0.0,3.06
4,BRUNELLO CUCINELLI,5427121,KTT0101899,63,3.27,4.14,0.0,3.27
...,...,...,...,...,...,...,...,...
1519147,RONI BLANSHAY,4135616,KTT0102994,46,0.00,0.00,0.0,0.00
1519148,SANDRO,5062306,KTT0100278,51,3.10,0.00,0.0,3.10
1519149,CELINE,2149202,KTT0100278,46,3.10,3.06,0.0,3.10
1519150,TRUE RELIGION,4159125,KTT0101901,44,3.30,0.00,0.0,3.30


In [10]:
df_gucci = mov_driver.read("""
select distinct
    item_id,
    first_price
from StagingArea.DM.PromoJournals

where 
    price_group_id in ('ЦУМ') and 
    journal_id = 'П000100849'
""")
df_gucci

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,item_id,first_price
0,НЕ00816095,132480.0
1,5484187,99360.0
2,НЕ00818893,285120.0
3,5133259,51840.0
4,НЕ00845684,31680.0
...,...,...
7275,НЕ00843545,34560.0
7276,5453617,403200.0
7277,НЕ00845115,56160.0
7278,НЕ00829742,79200.0


In [11]:
df_hugo = pd.read_excel(HUGO_BOSS_RRP_PATH)
df_hugo

,Price List,Material,Recom. Retail Price,Season Indicator
0,R1,50385164,95000,W23
1,R1,50470011,208000,W23
2,R1,50460298,182500,W23
3,R1,50426602,208000,W23
4,R1,50426590,247000,W23
...,...,...,...,...
13317,Bogner new,86792721,37700,W23
13318,Bogner new,11697985,50700,W23
13319,Bogner new,31654815,215500,W23
13320,Bogner new,86326937,35100,W23


In [12]:
df_costs_mus = df_costs.merge(df_mus, on=["itemid"], how="inner")
df_costs_mus

,Марка,Субмарка_x,itemid,_ktt1,_ktt2,_ktt3,_ktt3id_x,analog_code,axarticle,vendor_type,...,seasonid,cost_price_delivery_eur,cost_price_delivery_rur,Субмарка_y,_ktt3id_y,seasonkey_y,mu_old,mu_current,mu_article,mu
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,...,SS 2022,28.0,2393.0500,KISSY KISSY,KTT0201803,67,2.60,2.60,0.0,2.60
1,Givenchy,Givenchy,5570765,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,bm713q/3y6b,BM713Q3Y6B,ВнПОСТ_ЦУМ,...,SS 2021,204.0,20782.0300,GIVENCHY,KTT0101900,65,2.90,2.90,0.0,2.90
2,Armani,Armani Junior,1881322,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,M4B062H,M4B06/2H/12A-16A,РПОСТ_ЦУМ,...,SS 2010,187.5,16195.0000,ARMANI JUNIOR,KTT0201647,32,2.50,0.00,0.0,2.50
3,MONNALISA,Monnalisa Bebe,4230284,Товары для детей,Товары для новорожденных,Одежда,KTT0201758,314704/4914,314704/4914,ВнПОСТ_ЦУМ,...,FW 14/15,38.5,6994.9990,MONNALISA BEBE,KTT0201758,47,2.84,2.60,0.0,2.84
4,John Galliano Junior,John Galliano Junior,4029528,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,AWR/M1,00WAWR/JAAM1/10-14,РПОСТ_ЦУМ,...,SS 2012,56.9,6588.0000,JOHN GALLIANO JUNIOR,KTT0201647,41,0.00,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519146,Firetrap,Firetrap,1831698,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,fgdr114a/skull_raiser,FGDR114A/SKULL_RAISER,РПОСТ_ЦУМ,...,FW 09/10,14.0,0.0000,FIRETRAP,KTT0101900,34,0.00,0.00,0.0,0.00
1519147,Acne,Acne Studios,5111696,Товары для женщин,Аксессуары,Головные уборы,KTT0100662,17a164/hoy hat chunky,17A164,ВнПОСТ_ЦУМ,...,FW 16/17,46.0,4078.1600,ACNE STUDIOS,KTT0100662,53,3.12,3.12,0.0,3.12
1519148,Two women in the world,Two Women In The World,5093093,Товары для женщин,Одежда,Одежда джинсовая,KTT0101901,texsana lc4mc,TEXANA/LC4MC,ВнПОСТ_ЦУМ,...,FW 16/17,68.0,6530.5500,TWO WOMEN IN THE WORLD,KTT0101901,53,2.60,2.60,0.0,2.60
1519149,Moschino,Moschino,5001447,Товары для мужчин,Одежда,"Сорочки, рубашки",KTT0103078,0206/5261,J0206/5261,ВнПОСТ_ЦУМ,...,FW 15/16,131.0,10.0000,MOSCHINO,KTT0103078,50,2.70,2.92,0.0,2.70


In [13]:
df_costs_mus["analog_lower"] = df_costs_mus["analog_code"].str.lower()
df_costs_mus_base = df_costs_mus.merge(df_base, how="left", left_on=["analog_lower", "_ktt3id_x"], right_on=["analog_code", "ktt3id"])
df_costs_mus_base

,Марка,Субмарка_x,itemid,_ktt1,_ktt2,_ktt3,_ktt3id_x,analog_code_x,axarticle,vendor_type,...,mu_old,mu_current,mu_article,mu,analog_lower,analog_code_y,ktt3id,base_cost,season,Delivery PCS
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,...,2.60,2.60,0.0,2.60,kbq07089n.,NaN,NaN,NaN,NaN,NaN
1,Givenchy,Givenchy,5570765,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,bm713q/3y6b,BM713Q3Y6B,ВнПОСТ_ЦУМ,...,2.90,2.90,0.0,2.90,bm713q/3y6b,NaN,NaN,NaN,NaN,NaN
2,Armani,Armani Junior,1881322,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,M4B062H,M4B06/2H/12A-16A,РПОСТ_ЦУМ,...,2.50,0.00,0.0,2.50,m4b062h,NaN,NaN,NaN,NaN,NaN
3,MONNALISA,Monnalisa Bebe,4230284,Товары для детей,Товары для новорожденных,Одежда,KTT0201758,314704/4914,314704/4914,ВнПОСТ_ЦУМ,...,2.84,2.60,0.0,2.84,314704/4914,NaN,NaN,NaN,NaN,NaN
4,John Galliano Junior,John Galliano Junior,4029528,Товары для детей,Товары для мальчиков,Одежда,KTT0201647,AWR/M1,00WAWR/JAAM1/10-14,РПОСТ_ЦУМ,...,0.00,0.00,0.0,0.00,awr/m1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519146,Firetrap,Firetrap,1831698,Товары для мужчин,Одежда,Одежда джерси,KTT0101900,fgdr114a/skull_raiser,FGDR114A/SKULL_RAISER,РПОСТ_ЦУМ,...,0.00,0.00,0.0,0.00,fgdr114a/skull_raiser,NaN,NaN,NaN,NaN,NaN
1519147,Acne,Acne Studios,5111696,Товары для женщин,Аксессуары,Головные уборы,KTT0100662,17a164/hoy hat chunky,17A164,ВнПОСТ_ЦУМ,...,3.12,3.12,0.0,3.12,17a164/hoy hat chunky,NaN,NaN,NaN,NaN,NaN
1519148,Two women in the world,Two Women In The World,5093093,Товары для женщин,Одежда,Одежда джинсовая,KTT0101901,texsana lc4mc,TEXANA/LC4MC,ВнПОСТ_ЦУМ,...,2.60,2.60,0.0,2.60,texsana lc4mc,NaN,NaN,NaN,NaN,NaN
1519149,Moschino,Moschino,5001447,Товары для мужчин,Одежда,"Сорочки, рубашки",KTT0103078,0206/5261,J0206/5261,ВнПОСТ_ЦУМ,...,2.70,2.92,0.0,2.70,0206/5261,NaN,NaN,NaN,NaN,NaN


In [14]:
df_costs_mus_base.fillna(0, inplace=True)

In [15]:
esb_driver = SQLDriver("EasyBuying", "easybuying-tsum")
df_base_mus_current = esb_driver.read(
    f"""
    SELECT distinct
        ktm.Name AS sbrand,
        ktt.IdAxapta AS ktt_id,
        ktt.NodeLevel AS level,
        mu_value.Value AS mu,
        seasons.SerialNumber AS season_key,
		seasons.Name

    FROM [EasyBuying].[dbo].[MarkupBase] AS mu_base

    LEFT JOIN dbo.MarkupBaseValue AS mu_value ON mu_base.ID = mu_value.MarkupBaseID
    LEFT JOIN dbo.Season AS seasons ON mu_base.SeasonID = seasons.ID
    LEFT JOIN dbo.KTM AS ktm ON mu_base.SubBrandID = ktm.SubBrandID
    LEFT JOIN dbo.KTT AS ktt ON mu_base.KttID = ktt.ID
	LEFT JOIN dbo.KtmTree AS ktm_tree ON ktm.ID = ktm_tree.ID
    LEFT JOIN dbo.PriceGroup AS pg on mu_value.PricegroupID = pg.ID

    WHERE --mu_base.IsDeleted = 0 
    --AND mu_value.IsDeleted = 0
    mu_value.Value IS NOT NULL
    AND mu_value.Value > 0
    AND mu_base.SubBrandID IS NOT NULL
    --AND KttID IS NOT NULL
	AND pg.Name = 'РОЗН'
    AND seasons.SerialNumber <= 35
    """
)
df_base_mus_current

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,sbrand,ktt_id,level,mu,season_key,Name
0,None,None,NaN,3.200,34,SS 2023
1,None,None,NaN,3.200,35,FW 23/24
2,None,None,NaN,3.500,26,SS 2019
3,None,None,NaN,4.500,33,FW 22/23
4,None,None,NaN,5.376,32,SS 2022
...,...,...,...,...,...,...
138395,Триумф Норд,KTT0100005,2.0,2.500,30,SS 2021
138396,Триумф Норд,KTT0100005,2.0,2.500,32,SS 2022
138397,Триумф Норд,KTT0100005,2.0,2.500,33,FW 22/23
138398,Триумф Норд,KTT0100005,2.0,2.500,34,SS 2023


In [16]:
df_base_mus_current.replace("None", 0, inplace=True)
df_base_mus_current.fillna(0, inplace=True)
df_base_mus_current = df_base_mus_current[~df_base_mus_current.Name.isin(["SS 2023 Outlet", "FW 23/24 Outlet"])]

In [17]:
df_ktt1 = mov_driver.read(
    """
    SELECT DISTINCT Level1ID AS ktt_id, Level1Data, Level2Data, Level3Data, Level3ID AS ktt3_id
    FROM dbo.Item_KTT
    """
)

df_ktt2 = mov_driver.read(
    """
    SELECT DISTINCT Leve2ID AS ktt_id, Level1Data, Level2Data, Level3Data, Level3ID AS ktt3_id
    FROM dbo.Item_KTT
    """
)

df_ktt3 = mov_driver.read(
    """
    SELECT DISTINCT Level3ID AS ktt_id, Level1Data, Level2Data, Level3Data, Level3ID AS ktt3_id
    FROM dbo.Item_KTT
    """
)

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
df_ktt = pd.concat([df_base_mus_current.merge(df_ktt1, how="inner"), df_base_mus_current.merge(df_ktt2, how="inner"), df_base_mus_current.merge(df_ktt3, how="inner")]) 
df_ktt

,sbrand,ktt_id,level,mu,season_key,Name,Level1Data,Level2Data,Level3Data,ktt3_id
0,0,KTT0100009,1.0,2.90,18,SS 2015,Promo,Atrium,Аромалампа,KTT0100158
1,0,KTT0100009,1.0,2.90,18,SS 2015,Promo,Atrium,банка,KTT0100183
2,0,KTT0100009,1.0,2.90,18,SS 2015,Promo,Atrium,Блюдо,KTT0100283
3,0,KTT0100009,1.0,2.90,18,SS 2015,Promo,Atrium,Ваза,KTT0100546
4,0,KTT0100009,1.0,2.90,18,SS 2015,Promo,Atrium,Ведро д/шампанского,KTT0100565
...,...,...,...,...,...,...,...,...,...,...
108914,Ralph Lauren Home,KTT0100004,2.0,3.10,23,FW 17/18,Promo,China,China,KTT0100004
108915,Valentino,KTT0202280,3.0,2.95,30,SS 2021,Товары для женщин,Аксессуары,Перчатки защитные,KTT0202280
108916,Valentino,KTT0202280,3.0,2.95,31,FW 21/22,Товары для женщин,Аксессуары,Перчатки защитные,KTT0202280
108917,Valentino Garavani,KTT0202280,3.0,2.95,30,SS 2021,Товары для женщин,Аксессуары,Перчатки защитные,KTT0202280


In [19]:
df_mu_sbrand = df_base_mus_current.copy()
df_mu_sbrand = df_mu_sbrand[df_mu_sbrand.level==0]
df_mu_sbrand = df_mu_sbrand[["sbrand", "level", "mu", "season_key", "Name"]]
df_mu_sbrand["key"] = 0
df_ktt3["key"] = 0
df_mu_sbrand = df_ktt3.merge(df_mu_sbrand, how="inner")
df_mu_sbrand

,ktt_id,Level1Data,Level2Data,Level3Data,ktt3_id,key,sbrand,level,mu,season_key,Name
0,KTT0100003,Товары для детей,Канцелярские товары,CD-диск,KTT0100003,0,0,0.0,3.200,34,SS 2023
1,KTT0100003,Товары для детей,Канцелярские товары,CD-диск,KTT0100003,0,0,0.0,3.200,35,FW 23/24
2,KTT0100003,Товары для детей,Канцелярские товары,CD-диск,KTT0100003,0,0,0.0,3.500,26,SS 2019
3,KTT0100003,Товары для детей,Канцелярские товары,CD-диск,KTT0100003,0,0,0.0,4.500,33,FW 22/23
4,KTT0100003,Товары для детей,Канцелярские товары,CD-диск,KTT0100003,0,0,0.0,5.376,32,SS 2022
...,...,...,...,...,...,...,...,...,...,...,...
1831405,KTT0302684,Косметика,Для детей,Для детей,KTT0302684,0,Музей Янтаря,0.0,3.300,24,SS 2018
1831406,KTT0302684,Косметика,Для детей,Для детей,KTT0302684,0,Музей Янтаря,0.0,4.000,25,FW 18/19
1831407,KTT0302684,Косметика,Для детей,Для детей,KTT0302684,0,Павлопосадские платки,0.0,2.000,24,SS 2018
1831408,KTT0302684,Косметика,Для детей,Для детей,KTT0302684,0,Снег Идёт,0.0,2.000,25,FW 18/19


In [20]:
df_base_mu_t = pd.concat([df_ktt, df_mu_sbrand])

In [21]:
df_base_mu_t.query("ktt3_id == 'KTT0102250' and sbrand == 'Missoni'")
df_base_mu_t.key = df_base_mu_t.key.fillna(0)
df_base_mu_t

,sbrand,ktt_id,level,mu,season_key,Name,Level1Data,Level2Data,Level3Data,ktt3_id,key
0,0,KTT0100009,1.0,2.9,18,SS 2015,Promo,Atrium,Аромалампа,KTT0100158,0.0
1,0,KTT0100009,1.0,2.9,18,SS 2015,Promo,Atrium,банка,KTT0100183,0.0
2,0,KTT0100009,1.0,2.9,18,SS 2015,Promo,Atrium,Блюдо,KTT0100283,0.0
3,0,KTT0100009,1.0,2.9,18,SS 2015,Promo,Atrium,Ваза,KTT0100546,0.0
4,0,KTT0100009,1.0,2.9,18,SS 2015,Promo,Atrium,Ведро д/шампанского,KTT0100565,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1831405,Музей Янтаря,KTT0302684,0.0,3.3,24,SS 2018,Косметика,Для детей,Для детей,KTT0302684,0.0
1831406,Музей Янтаря,KTT0302684,0.0,4.0,25,FW 18/19,Косметика,Для детей,Для детей,KTT0302684,0.0
1831407,Павлопосадские платки,KTT0302684,0.0,2.0,24,SS 2018,Косметика,Для детей,Для детей,KTT0302684,0.0
1831408,Снег Идёт,KTT0302684,0.0,2.0,25,FW 18/19,Косметика,Для детей,Для детей,KTT0302684,0.0


In [22]:
df_base_mu = df_base_mu_t.sort_values(["season_key", "level"], ascending=False)
df_base_mu.drop_duplicates(subset=["sbrand", "ktt3_id"], inplace=True)
df_base_mu = df_base_mu[["sbrand", "ktt3_id", "mu"]]
df_base_mu.columns = ["sbrand", "ktt3id", "season_mu"]
df_base_mu

,sbrand,ktt3id,season_mu
70312,Emporio Armani,KTT0101881,2.4
70759,Emporio Armani,KTT0101883,2.4
71607,Z ZEGNA,KTT0102731,3.1
72365,Berluti,KTT0100278,2.9
75825,Corneliani,KTT0101307,4.0
...,...,...,...
6012440,Marc by Marc Jacobs,KTT0200638,2.8
6012441,Marc by Marc Jacobs,KTT0200793,2.8
6012442,Marc by Marc Jacobs,KTT0103334,2.8
6012443,Marc by Marc Jacobs,KTT0103677,2.8


In [23]:
df_costs_mus_base_season = df_costs_mus_base.merge(df_base_mu, how="inner", left_on=["Субмарка_x", "_ktt3id_x"], right_on=["sbrand", "ktt3id"])
df_costs_mus_base_season

,Марка,Субмарка_x,itemid,_ktt1,_ktt2,_ktt3,_ktt3id_x,analog_code_x,axarticle,vendor_type,...,mu,analog_lower,analog_code_y,ktt3id_x,base_cost,season,Delivery PCS,sbrand,ktt3id_y,season_mu
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,...,2.60,kbq07089n.,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60
1,Kissy Kissy,Kissy Kissy,5217747,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,F17416V-205ST/WINTER MIX,F17146V-205ST,ВнПОСТ_ЦУМ,...,2.60,f17416v-205st/winter mix,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60
2,Kissy Kissy,Kissy Kissy,5364148,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ00232N/DINOSAUR ROARR - 204,KBQ00232N,ВнПОСТ_ЦУМ,...,2.60,kbq00232n/dinosaur roarr - 204,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60
3,Kissy Kissy,Kissy Kissy,4069510,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,04P/F12172,F12172/04P,РПОСТ_ЦУМ,...,2.60,04p/f12172,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60
4,Kissy Kissy,Kissy Kissy,5132183,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,S17410-04P,S17410-04P,ВнПОСТ_ЦУМ,...,2.60,s17410-04p,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296023,Simone Rocha,Simone Rocha,5270881,Товары для женщин,Обувь,Сабо,KTT0102825,sld8/0744,SLD8/0744,ВнПОСТ_ЦУМ,...,2.45,sld8/0744,0,0,0.0,0,0,Simone Rocha,KTT0102825,2.45
1296024,PUBLIC SCHOOL,Public School,1788078,Товары для мужчин,Одежда,"Пиджаки, жилеты",KTT0102122,ps-fw14,PS-FW14,ВнПОСТ_ЦУМ,...,2.70,ps-fw14,0,0,0.0,0,0,Public School,KTT0102122,2.70
1296025,Fendi,Fendi,5484005,Товары для женщин,Бижутерия,Серьги,KTT0102994,8ag797/ab0t,8AG797/AB0T,ЮрЛица,...,3.12,8ag797/ab0t,0,0,0.0,0,0,Fendi,KTT0102994,3.00
1296026,RARE,Rare,5473750,Товары для мужчин,Обувь,Лоуферы,KTT0201536,ru3253/varsavia lux,RU3253/VARSAVIA LUX,ВнПОСТ_ЦУМ,...,2.70,ru3253/varsavia lux,0,0,0.0,0,0,Rare,KTT0201536,2.70


In [24]:
def calc_price(row):
    cost_price = row.cost_price_delivery_eur
    mu = row.season_mu
    if row.mu_article > 0:
        mu = row.mu_article
    if row.base_cost > 0:
        cost_price = row.base_cost
    temp_rate = EXCHANGE_RATE
    if row.vendor_type == "РПОСТ_ЦУМ":
        if row.cost_price_delivery_rur / 20 <= row.cost_price_delivery_eur:
            return temp_rate * cost_price * mu
        temp_rate = 1
        cost_price = row.cost_price_delivery_rur
    return temp_rate * cost_price * mu

In [25]:
df_costs_mus_base_season["calculated_price"] = df_costs_mus_base_season.apply(lambda x: calc_price(x), axis=1)
df_costs_mus_base_season

,Марка,Субмарка_x,itemid,_ktt1,_ktt2,_ktt3,_ktt3id_x,analog_code_x,axarticle,vendor_type,...,analog_lower,analog_code_y,ktt3id_x,base_cost,season,Delivery PCS,sbrand,ktt3id_y,season_mu,calculated_price
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,...,kbq07089n.,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60,6221.93000
1,Kissy Kissy,Kissy Kissy,5217747,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,F17416V-205ST/WINTER MIX,F17146V-205ST,ВнПОСТ_ЦУМ,...,f17416v-205st/winter mix,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60,6721.00000
2,Kissy Kissy,Kissy Kissy,5364148,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ00232N/DINOSAUR ROARR - 204,KBQ00232N,ВнПОСТ_ЦУМ,...,kbq00232n/dinosaur roarr - 204,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60,5791.50000
3,Kissy Kissy,Kissy Kissy,4069510,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,04P/F12172,F12172/04P,РПОСТ_ЦУМ,...,04p/f12172,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60,4160.01300
4,Kissy Kissy,Kissy Kissy,5132183,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,S17410-04P,S17410-04P,ВнПОСТ_ЦУМ,...,s17410-04p,0,0,0.0,0,0,Kissy Kissy,KTT0201803,2.60,5005.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296023,Simone Rocha,Simone Rocha,5270881,Товары для женщин,Обувь,Сабо,KTT0102825,sld8/0744,SLD8/0744,ВнПОСТ_ЦУМ,...,sld8/0744,0,0,0.0,0,0,Simone Rocha,KTT0102825,2.45,40576.37815
1296024,PUBLIC SCHOOL,Public School,1788078,Товары для мужчин,Одежда,"Пиджаки, жилеты",KTT0102122,ps-fw14,PS-FW14,ВнПОСТ_ЦУМ,...,ps-fw14,0,0,0.0,0,0,Public School,KTT0102122,2.70,22227.09390
1296025,Fendi,Fendi,5484005,Товары для женщин,Бижутерия,Серьги,KTT0102994,8ag797/ab0t,8AG797/AB0T,ЮрЛица,...,8ag797/ab0t,0,0,0.0,0,0,Fendi,KTT0102994,3.00,75570.00000
1296026,RARE,Rare,5473750,Товары для мужчин,Обувь,Лоуферы,KTT0201536,ru3253/varsavia lux,RU3253/VARSAVIA LUX,ВнПОСТ_ЦУМ,...,ru3253/varsavia lux,0,0,0.0,0,0,Rare,KTT0201536,2.70,46035.00000


In [26]:
df_gucci = mov_driver.read("""SELECT DISTINCT
      [item_id]
      , [first_price]
  FROM [StagingArea].[DM].[PromoJournals]

  Where [price_group_id] in ('ЦУМ') and [journal_id] = 'П000100849'""")
df_boss = pd.read_excel(r"\\mercury.ru\dfspublic$\ЦУМ\Аналитический отдел\_by Names\~ADSmirnov\Ценообразование\Hugo Boss РРЦ.xlsx")
df_gucci.columns = ["itemid", "gucci_price"]
df_boss.columns = ["price_list", "Material", "boss_price", "boss_season"]
df_costs_mus_base_season_b_g = df_costs_mus_base_season.merge(df_boss, how="left", left_on=["axarticle"], right_on=["Material"])
df_costs_mus_base_season_b_g = df_costs_mus_base_season_b_g.merge(df_gucci, how='left')
df_costs_mus_base_season_b_g

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Марка,Субмарка_x,itemid,_ktt1,_ktt2,_ktt3,_ktt3id_x,analog_code_x,axarticle,vendor_type,...,Delivery PCS,sbrand,ktt3id_y,season_mu,calculated_price,price_list,Material,boss_price,boss_season,gucci_price
0,Kissy Kissy,Kissy Kissy,НЕ00765016,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ07089N.,KBQ07089N.,РПОСТ_ЦУМ,...,0,Kissy Kissy,KTT0201803,2.60,6221.93000,NaN,NaN,NaN,NaN,NaN
1,Kissy Kissy,Kissy Kissy,5217747,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,F17416V-205ST/WINTER MIX,F17146V-205ST,ВнПОСТ_ЦУМ,...,0,Kissy Kissy,KTT0201803,2.60,6721.00000,NaN,NaN,NaN,NaN,NaN
2,Kissy Kissy,Kissy Kissy,5364148,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,KBQ00232N/DINOSAUR ROARR - 204,KBQ00232N,ВнПОСТ_ЦУМ,...,0,Kissy Kissy,KTT0201803,2.60,5791.50000,NaN,NaN,NaN,NaN,NaN
3,Kissy Kissy,Kissy Kissy,4069510,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,04P/F12172,F12172/04P,РПОСТ_ЦУМ,...,0,Kissy Kissy,KTT0201803,2.60,4160.01300,NaN,NaN,NaN,NaN,NaN
4,Kissy Kissy,Kissy Kissy,5132183,Товары для детей,Товары для новорожденных,"Домашняя, пляжная одежда, белье",KTT0201803,S17410-04P,S17410-04P,ВнПОСТ_ЦУМ,...,0,Kissy Kissy,KTT0201803,2.60,5005.00000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296023,Simone Rocha,Simone Rocha,5270881,Товары для женщин,Обувь,Сабо,KTT0102825,sld8/0744,SLD8/0744,ВнПОСТ_ЦУМ,...,0,Simone Rocha,KTT0102825,2.45,40576.37815,NaN,NaN,NaN,NaN,NaN
1296024,PUBLIC SCHOOL,Public School,1788078,Товары для мужчин,Одежда,"Пиджаки, жилеты",KTT0102122,ps-fw14,PS-FW14,ВнПОСТ_ЦУМ,...,0,Public School,KTT0102122,2.70,22227.09390,NaN,NaN,NaN,NaN,NaN
1296025,Fendi,Fendi,5484005,Товары для женщин,Бижутерия,Серьги,KTT0102994,8ag797/ab0t,8AG797/AB0T,ЮрЛица,...,0,Fendi,KTT0102994,3.00,75570.00000,NaN,NaN,NaN,NaN,NaN
1296026,RARE,Rare,5473750,Товары для мужчин,Обувь,Лоуферы,KTT0201536,ru3253/varsavia lux,RU3253/VARSAVIA LUX,ВнПОСТ_ЦУМ,...,0,Rare,KTT0201536,2.70,46035.00000,NaN,NaN,NaN,NaN,NaN


In [27]:
df_costs_mus_base_season_b_g.loc[df_costs_mus_base_season_b_g.gucci_price > 0, "calculated_price"] = df_costs_mus_base_season_b_g.gucci_price
df_costs_mus_base_season_b_g.loc[df_costs_mus_base_season_b_g.boss_price > 0, "calculated_price"] = df_costs_mus_base_season_b_g.boss_price

In [28]:
df_costs_mus_base_season_b_g_c = df_costs_mus_base_season_b_g[["itemid","Марка", "Субмарка_x", "seasonid", "calculated_price"]]
df_costs_mus_base_season_b_g_c.columns = ["ItemID", "Марка", "Субмарка", "LastSeason", "CalculatedPrice"]
df_costs_mus_base_season_b_g_c.drop_duplicates(["ItemID", "Марка", "Субмарка", "LastSeason"], inplace=True)
df_costs_mus_base_season_b_g_c

,ItemID,Марка,Субмарка,LastSeason,CalculatedPrice
0,НЕ00765016,Kissy Kissy,Kissy Kissy,SS 2022,6221.93000
1,5217747,Kissy Kissy,Kissy Kissy,FW 17/18,6721.00000
2,5364148,Kissy Kissy,Kissy Kissy,FW 18/19,5791.50000
3,4069510,Kissy Kissy,Kissy Kissy,FW 12/13,4160.01300
4,5132183,Kissy Kissy,Kissy Kissy,SS 2017,5005.00000
...,...,...,...,...,...
1296023,5270881,Simone Rocha,Simone Rocha,SS 2018,40576.37815
1296024,1788078,PUBLIC SCHOOL,Public School,FW 08/09,22227.09390
1296025,5484005,Fendi,Fendi,SS 2020,75570.00000
1296026,5473750,RARE,Rare,SS 2020,46035.00000


In [29]:
df_articles = dbd_driver.read("""
select distinct Item.ItemID, ICS.ColorID, Item.AnalogCode, ICS.ColorAnalogCode ColorAnalogCode, Item.Article, Item.Fabric,
KTT.Level1Data ktt1, KTT.Level2Data ktt2, KTT.Level3Data ktt3, KTT.Level4Data ktt4
from MOVOfGoodsDynamic.dbo.Item_ICS ICS
left join MOVOfGoodsDynamic.dwh.DIM_Item as item2 on item2.ItemKey = ICS.ItemKey
LEFT JOIN MOVOfGoodsDynamic.dbo.Item	    AS Item     ON Item.ItemKey = ICS.ItemKey
LEFT JOIN MOVOfGoodsDynamic.dbo.Item_KTT      AS KTT      ON CAST(RIGHT(ktt.Level7ID, 7) AS INT) = item2.kttkey


""")
df_articles

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ItemID,ColorID,AnalogCode,ColorAnalogCode,Article,Fabric,ktt1,ktt2,ktt3,ktt4
0,НЕ00439036,"REAL LILAC,REAL",DB3539,,DB3539,POD-S3.1 PK,Товары для мужчин,Обувь,Кроссовки,Кроссовки
1,4250426,Разноцв_MIKAD0,OXFORD COURT MIKADO/T04,1B1,0XF0RD C0URT/SH0XC03 T04,T04,Товары для мужчин,"Домашняя, пляжная одежда",Одежда пляжная,Плавки
2,5235525,Черный,WH7130-319MXBLK/BRIGHTON,MAXBLK,WH7130-319MXBLK,BRIGHTON,Товары для женщин,Одежда,Одежда джинсовая,Шорты джинсовые
3,1719600,Розовый_20,8194M/~~,20,E/8194/E1370,~~,Товары для детей,Товары для девочек,Одежда,Платье
4,5449168,Красный_0H05,WESTIE/001-4000646,0H05,0014000646/01/20-23,001-4000646,Товары для детей,Товары для мальчиков,Обувь,Тапочки
...,...,...,...,...,...,...,...,...,...,...
2959433,4182986,С-серый_190,DB718H/5201175,190,DB718H/5201175,5201175,Товары для женщин,Одежда,Блузы,Блуза
2959434,НЕ00184454,88N,I200CG2Z.00FVN,,I200CG2Z.00FVN,,Товары для мужчин,"Домашняя, пляжная одежда",Белье,Трусы-боксеры
2959435,1366247,Разноцв,1913D003,,1913D003,,Прочее,Упаковка,Упаковка,Упаковка
2959436,НЕ00137505,,Hibiscus сем,,Hibiscus сем,,Товары для дома,Товары для Дома,SALE Товары для дома и досуга,SALE Товары для дома и досуга


In [30]:
df_articles.drop_duplicates(["ItemID", "ColorID"], inplace=True)

In [31]:
df_articles2 = df_articles.merge(df_costs_mus_base_season_b_g_c, how="inner")
df_articles2

,ItemID,ColorID,AnalogCode,ColorAnalogCode,Article,Fabric,ktt1,ktt2,ktt3,ktt4,Марка,Субмарка,LastSeason,CalculatedPrice
0,НЕ00439036,"REAL LILAC,REAL",DB3539,,DB3539,POD-S3.1 PK,Товары для мужчин,Обувь,Кроссовки,Кроссовки,Adidas,Adidas Originals,SS 2019,9123.348000
1,4250426,Разноцв_MIKAD0,OXFORD COURT MIKADO/T04,1B1,0XF0RD C0URT/SH0XC03 T04,T04,Товары для мужчин,"Домашняя, пляжная одежда",Одежда пляжная,Плавки,Robinson Les Bains,Robinson Les Bains,FW 14/15,21164.000000
2,5235525,Черный,WH7130-319MXBLK/BRIGHTON,MAXBLK,WH7130-319MXBLK,BRIGHTON,Товары для женщин,Одежда,Одежда джинсовая,Шорты джинсовые,RTA,RTA,SS 2018,52291.800000
3,5449168,Красный_0H05,WESTIE/001-4000646,0H05,0014000646/01/20-23,001-4000646,Товары для детей,Товары для мальчиков,Обувь,Тапочки,Naturino,Naturino,FW 19/20,6336.000000
4,5449168,Синий_0C01,WESTIE/001-4000646,0C01,0014000646/01/20-23,001-4000646,Товары для детей,Товары для мальчиков,Обувь,Тапочки,Naturino,Naturino,FW 19/20,6336.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720320,5455407,Коричн_TAN,19FWMDASTNDMED,TAN TN,19FWMDASTNDMED,,Товары для женщин,Сумки,Сумки обычные,Сумка,By Far,By Far,SS 2021,97065.349068
1720321,1700112,Серебр_040,HUGO 708/10108542,040,50119292/HUGO 708,10108542,Товары для мужчин,Одежда,Одежда джинсовая,Джинсы,HUGO BOSS,HUGO,SS 2008,6105.120000
1720322,4234806,Белый_10000,3935796,10000,3935796,,Товары для детей,Товары для новорожденных,Одежда,Одежда джерси,Burberry,Burberry,FW 14/15,7484.400000
1720323,5055506,Черный_T9000,CP1300/T13940,T9000,CP1300/T13940,T13940,Товары для женщин,Одежда,Одежда верхняя,Плащи,Tom Ford,Tom Ford,SS 2016,451770.000000


In [32]:
df_articles2.drop_duplicates(["ItemID", "ColorID"], inplace=True)
df_articles2

,ItemID,ColorID,AnalogCode,ColorAnalogCode,Article,Fabric,ktt1,ktt2,ktt3,ktt4,Марка,Субмарка,LastSeason,CalculatedPrice
0,НЕ00439036,"REAL LILAC,REAL",DB3539,,DB3539,POD-S3.1 PK,Товары для мужчин,Обувь,Кроссовки,Кроссовки,Adidas,Adidas Originals,SS 2019,9123.348000
1,4250426,Разноцв_MIKAD0,OXFORD COURT MIKADO/T04,1B1,0XF0RD C0URT/SH0XC03 T04,T04,Товары для мужчин,"Домашняя, пляжная одежда",Одежда пляжная,Плавки,Robinson Les Bains,Robinson Les Bains,FW 14/15,21164.000000
2,5235525,Черный,WH7130-319MXBLK/BRIGHTON,MAXBLK,WH7130-319MXBLK,BRIGHTON,Товары для женщин,Одежда,Одежда джинсовая,Шорты джинсовые,RTA,RTA,SS 2018,52291.800000
3,5449168,Красный_0H05,WESTIE/001-4000646,0H05,0014000646/01/20-23,001-4000646,Товары для детей,Товары для мальчиков,Обувь,Тапочки,Naturino,Naturino,FW 19/20,6336.000000
4,5449168,Синий_0C01,WESTIE/001-4000646,0C01,0014000646/01/20-23,001-4000646,Товары для детей,Товары для мальчиков,Обувь,Тапочки,Naturino,Naturino,FW 19/20,6336.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720320,5455407,Коричн_TAN,19FWMDASTNDMED,TAN TN,19FWMDASTNDMED,,Товары для женщин,Сумки,Сумки обычные,Сумка,By Far,By Far,SS 2021,97065.349068
1720321,1700112,Серебр_040,HUGO 708/10108542,040,50119292/HUGO 708,10108542,Товары для мужчин,Одежда,Одежда джинсовая,Джинсы,HUGO BOSS,HUGO,SS 2008,6105.120000
1720322,4234806,Белый_10000,3935796,10000,3935796,,Товары для детей,Товары для новорожденных,Одежда,Одежда джерси,Burberry,Burberry,FW 14/15,7484.400000
1720323,5055506,Черный_T9000,CP1300/T13940,T9000,CP1300/T13940,T13940,Товары для женщин,Одежда,Одежда верхняя,Плащи,Tom Ford,Tom Ford,SS 2016,451770.000000


In [33]:
df_articles2 = df_articles2[["Марка", "Субмарка", "ktt1", "ktt2", "ktt3", "ktt4", "ItemID", "ColorID", "ColorAnalogCode", "AnalogCode", "Article", "Fabric", "LastSeason", "CalculatedPrice"]]

In [34]:
df_articles2 = df_articles2[df_articles2.CalculatedPrice > 0]

In [35]:
df_articles2["modified"] = date.today()

In [36]:
df_articles_old = dbd_driver2.read("""
    select 
        ItemID, 
        ColorID, 
        CalculatedPrice as old_price, 
        modified as old_modified                     
from SalesDevelopment.CalculatedPrice""")
df_articles_old

c:\Users\IMNedobezhkin\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ItemID,ColorID,old_price,old_modified
0,1589624,Черный,28242.0,2023-09-22
1,5379045,Т-синий_J16U,111787.0,2023-09-22
2,5460621,Черный_99,32120.0,2023-09-21
3,5156115,Черный_05051,273273.0,2023-09-21
4,1891299,Белый_A1000,22847.0,2023-09-21
...,...,...,...,...
838730,1855525,Черн-бел_1095,157465.0,2023-09-21
838731,2177579,Синий_019,111078.0,2023-09-21
838732,5206334,Фиолет,519750.0,2023-09-21
838733,1793208,Крем_/CREAM,21265.0,2023-09-21


In [37]:
df_articles2.CalculatedPrice = df_articles2.CalculatedPrice.astype(int)
df_articles_check = df_articles2.merge(df_articles_old, how="left")
df_articles_check.fillna(0, inplace=True)
df_articles_check.old_price = df_articles_check.old_price.astype(int)
df_articles_check.loc[df_articles_check.old_price == df_articles_check.CalculatedPrice, "modified"] = df_articles_check.old_modified
df_articles_check.drop("old_price", axis=1, inplace=True)
df_articles_check.drop("old_modified", axis=1, inplace=True)

In [38]:
df_articles_check

,Марка,Субмарка,ktt1,ktt2,ktt3,ktt4,ItemID,ColorID,ColorAnalogCode,AnalogCode,Article,Fabric,LastSeason,CalculatedPrice,modified
0,Adidas,Adidas Originals,Товары для мужчин,Обувь,Кроссовки,Кроссовки,НЕ00439036,"REAL LILAC,REAL",,DB3539,DB3539,POD-S3.1 PK,SS 2019,9123,2023-09-22
1,Robinson Les Bains,Robinson Les Bains,Товары для мужчин,"Домашняя, пляжная одежда",Одежда пляжная,Плавки,4250426,Разноцв_MIKAD0,1B1,OXFORD COURT MIKADO/T04,0XF0RD C0URT/SH0XC03 T04,T04,FW 14/15,21164,2023-09-22
2,RTA,RTA,Товары для женщин,Одежда,Одежда джинсовая,Шорты джинсовые,5235525,Черный,MAXBLK,WH7130-319MXBLK/BRIGHTON,WH7130-319MXBLK,BRIGHTON,SS 2018,52291,2023-09-22
3,Naturino,Naturino,Товары для детей,Товары для мальчиков,Обувь,Тапочки,5449168,Красный_0H05,0H05,WESTIE/001-4000646,0014000646/01/20-23,001-4000646,FW 19/20,6336,2023-09-22
4,Naturino,Naturino,Товары для детей,Товары для мальчиков,Обувь,Тапочки,5449168,Синий_0C01,0C01,WESTIE/001-4000646,0014000646/01/20-23,001-4000646,FW 19/20,6336,2023-09-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1720318,By Far,By Far,Товары для женщин,Сумки,Сумки обычные,Сумка,5455407,Коричн_TAN,TAN TN,19FWMDASTNDMED,19FWMDASTNDMED,,SS 2021,97065,2023-09-22
1720319,HUGO BOSS,HUGO,Товары для мужчин,Одежда,Одежда джинсовая,Джинсы,1700112,Серебр_040,040,HUGO 708/10108542,50119292/HUGO 708,10108542,SS 2008,6105,2023-09-22
1720320,Burberry,Burberry,Товары для детей,Товары для новорожденных,Одежда,Одежда джерси,4234806,Белый_10000,10000,3935796,3935796,,FW 14/15,7484,2023-09-22
1720321,Tom Ford,Tom Ford,Товары для женщин,Одежда,Одежда верхняя,Плащи,5055506,Черный_T9000,T9000,CP1300/T13940,CP1300/T13940,T13940,SS 2016,451770,2023-09-21


In [39]:
with dbd_driver2.conn_context() as c: 
    c.execute("DELETE FROM SalesDevelopment.CalculatedPrice")
    c.commit()


ktts = df_articles_check["ktt2"].unique()

for ktt in ktts:
    temp_df = df_articles_check[df_articles_check["ktt2"] == ktt]
    dbd_driver2.bulk_insert("SalesDevelopment", "CalculatedPrice", temp_df)